In [1]:
import numpy as np
import xarray as xr

In [2]:
import stmat


In [3]:
import dask

In [4]:
import pygeos
import geopandas as gpd
import shapely as sp
#import shapely.vectorized as spvec



Read file with polygon delimited geometeries in the assendelft region 

In [5]:
as_poly = gpd.read_file("/Users/eslt0101/Projects/REPOS/eScience/MoByLe/data/assendelft/assendelft_attributes_wgs84.shp")

In [6]:
as_poly['gewascode']

0         265
1         259
2        2014
3         265
4         259
         ... 
11464     265
11465     331
11466     331
11467     265
11468     331
Name: gewascode, Length: 11469, dtype: int64

This has a number of fields which we could use as the field of interest. For the time being we'll use 'gewascode'


Rather than loading a large observation subsetting and then using that, we'll generate synthetic raster data based on the extent covered by the geometry file.

First, retrieve extent and make a superset region

In [7]:
minx = as_poly['geometry'][0].representative_point().x
maxx = as_poly['geometry'][0].representative_point().x
miny = as_poly['geometry'][0].representative_point().y
maxy = as_poly['geometry'][0].representative_point().x

for i in range(len(as_poly)):
    x= as_poly['geometry'][i].representative_point().x
    y= as_poly['geometry'][i].representative_point().y
    if x <= minx:
        minx=x
    if x >= maxx:
        maxx=x
    if y <= miny:
        miny=y
    if y>= maxy:
        maxy=y
     



In [8]:
print(minx, maxx)
print(miny, maxy)

4.666600983608291 5.038156842391597
52.39466674333789 52.58735609848867


In [9]:
fx = (maxx - minx)/np.sqrt(maxx*minx)
fy = (maxy - miny)/np.sqrt(maxy*miny)

minx = (1-fx)*minx
miny = (1-fy)*miny
maxx= (1+fx)*maxx
maxy= (1 + fy)*maxy

In [10]:
miny

52.20233073607125

Create synthetic data 

First using dimensions as stated below

In [11]:
n_points = 10000
n_time = 3

synthetic h2ph data

In [12]:
stack = np.empty((n_points,0))
       
for i in range(3):
    slc=np.random.random((100,100))
    stack = np.append(stack, slc[0:100,0:100].reshape(n_points,1), axis=1)

print(stack)
print(stack.shape)


[[0.9008002  0.15390946 0.51951901]
 [0.25197449 0.46049996 0.63346729]
 [0.05808114 0.87354917 0.77006908]
 ...
 [0.71111553 0.12123751 0.19756597]
 [0.4899125  0.28412062 0.75192673]
 [0.37819324 0.79843008 0.17205713]]
(10000, 3)


synthetic coords

In [13]:
x = np.sort(np.random.random(100)*(maxx - minx) + minx)
y = np.sort(np.random.random(100)*(maxy - miny) + miny)


In [14]:
xx, yy = np.meshgrid(x,y)

In [15]:
xxreshape = xx.reshape(n_points,1)
yyreshape = yy.reshape(n_points,1)

In [16]:
xxreshape

array([[4.31288   ],
       [4.32970261],
       [4.33079089],
       ...,
       [5.39406394],
       [5.39779416],
       [5.41938476]])

In [17]:
stack_xx = np.empty((n_points,0))
stack_yy = np.empty((n_points,0))

for i in range(3):
    stack_xx = np.append(stack_xx, xxreshape, axis=1)
    stack_yy = np.append(stack_yy, yyreshape, axis=1)

print(stack_xx)
print(stack_xx.shape)


[[4.31288    4.31288    4.31288   ]
 [4.32970261 4.32970261 4.32970261]
 [4.33079089 4.33079089 4.33079089]
 ...
 [5.39406394 5.39406394 5.39406394]
 [5.39779416 5.39779416 5.39779416]
 [5.41938476 5.41938476 5.41938476]]
(10000, 3)


In [18]:
data_xr = xr.Dataset(data_vars=dict(h2ph=(["points","time"], stack), xv=(["points"],xxreshape.flatten()),yv=(["points"],yyreshape.flatten()),),
                     coords=dict(
                        points=np.arange(n_points),
                        time=np.arange(n_time),
                        ))

In [19]:
data_xr

<xarray.Dataset>
Dimensions:  (points: 10000, time: 3)
Coordinates:
  * points   (points) int64 0 1 2 3 4 5 6 ... 9993 9994 9995 9996 9997 9998 9999
  * time     (time) int64 0 1 2
Data variables:
    h2ph     (points, time) float64 0.9008 0.1539 0.5195 ... 0.7984 0.1721
    xv       (points) float64 4.313 4.33 4.331 4.369 ... 5.394 5.394 5.398 5.419
    yv       (points) float64 52.24 52.24 52.24 52.24 ... 52.77 52.77 52.77

In [20]:
data_xr = data_xr.stm.add_metadata({"name":"synthetic_test_stm_assendelft"})

In [21]:
data_xr['xv'][0:10]
data_xr['yv'][0:10]

<xarray.DataArray 'yv' (points: 10)>
array([52.23630109, 52.23630109, 52.23630109, 52.23630109, 52.23630109,
       52.23630109, 52.23630109, 52.23630109, 52.23630109, 52.23630109])
Coordinates:
  * points   (points) int64 0 1 2 3 4 5 6 7 8 9

In [22]:
data_xr['yv'].data


array([52.23630109, 52.23630109, 52.23630109, ..., 52.76931712,
       52.76931712, 52.76931712])

In [23]:
data_xr.xv.coords


Coordinates:
  * points   (points) int64 0 1 2 3 4 5 6 ... 9993 9994 9995 9996 9997 9998 9999

We now have synthetic SpaceTime matrix dataset in xarray format 

Create an empty data array of correct dimensions to save info to be retrieved in the dataset

In [24]:
field = xr.DataArray(coords=data_xr.xv.coords,name='gewas').astype(int)

add to dataset

In [25]:
data_xr['gewas'] = field

In [26]:
data_xr

<xarray.Dataset>
Dimensions:  (points: 10000, time: 3)
Coordinates:
  * points   (points) int64 0 1 2 3 4 5 6 ... 9993 9994 9995 9996 9997 9998 9999
  * time     (time) int64 0 1 2
Data variables:
    h2ph     (points, time) float64 0.9008 0.1539 0.5195 ... 0.7984 0.1721
    xv       (points) float64 4.313 4.33 4.331 4.369 ... 5.394 5.394 5.398 5.419
    yv       (points) float64 52.24 52.24 52.24 52.24 ... 52.77 52.77 52.77
    gewas    (points) int64 -9223372036854775808 ... -9223372036854775808
Attributes:
    name:     synthetic_test_stm_assendelft

In [27]:
data_xr['xv'][0:4]

<xarray.DataArray 'xv' (points: 4)>
array([4.31288   , 4.32970261, 4.33079089, 4.3692904 ])
Coordinates:
  * points   (points) int64 0 1 2 3

In [28]:
data_xr['xv'][[0,2]]

<xarray.DataArray 'xv' (points: 2)>
array([4.31288   , 4.33079089])
Coordinates:
  * points   (points) int64 0 2

Use pygeos vectorized matching operation, update dataframe in loop 

In [29]:
stmTree = pygeos.STRtree(pygeos.points(data_xr['xv'].data, data_xr['yv'].data))
as_poly_geoms = np.array([pygeos.from_shapely(geo) for geo in as_poly['geometry']])
ml = np.array(stmTree.query_bulk(as_poly_geoms,predicate='contains').T.tolist())
#this returns an array of two element arrays, the first entry is the positional index
#into the list of geometries being used to query the tree. the second is the positional index
#into the list of points for which the tree was constructed

#get list of unique matched query geometries
uids = np.unique(ml[:,0])
for i, uid in enumerate(uids):
    #identify all instances of points falling with the query geometry
    m = np.where(ml[:,0]==uid)[0]
    #retrieve indicees of these points (their position in the input used to create the tree)
    mid = ml[m,1]
    #update the fields correspondig to these indicees
    data_xr['gewas'].data[mid] = as_poly['gewascode'][uid]
    

In [30]:
data_xr

<xarray.Dataset>
Dimensions:  (points: 10000, time: 3)
Coordinates:
  * points   (points) int64 0 1 2 3 4 5 6 ... 9993 9994 9995 9996 9997 9998 9999
  * time     (time) int64 0 1 2
Data variables:
    h2ph     (points, time) float64 0.9008 0.1539 0.5195 ... 0.7984 0.1721
    xv       (points) float64 4.313 4.33 4.331 4.369 ... 5.394 5.394 5.398 5.419
    yv       (points) float64 52.24 52.24 52.24 52.24 ... 52.77 52.77 52.77
    gewas    (points) int64 -9223372036854775808 ... -9223372036854775808
Attributes:
    name:     synthetic_test_stm_assendelft

In [31]:
data_xr['gewas'].data.max()

2785

This demonstrates the basic workflow of polygon based data enrichment. It can, of course, also serve to filter a point set based on a polygon/geometry.

The above, however, considers a STM of a size that can fit into memory. While the assumption that the (set of) geometry(ies) to be queried fits into memory is reasonable, this certainly mayy not be the case for the STM. In the following we consider a chunked processing, that is also conducive to parallel execution. 

In the case of large STM with out-of-core calculation, the STM xarray object will be backed by chunked dask arrays. Therefore, we first convert our STM `data_xr` accordingly.

In [32]:
data2 = data_xr.copy(deep=True)

In [33]:
data2

<xarray.Dataset>
Dimensions:  (points: 10000, time: 3)
Coordinates:
  * points   (points) int64 0 1 2 3 4 5 6 ... 9993 9994 9995 9996 9997 9998 9999
  * time     (time) int64 0 1 2
Data variables:
    h2ph     (points, time) float64 0.9008 0.1539 0.5195 ... 0.7984 0.1721
    xv       (points) float64 4.313 4.33 4.331 4.369 ... 5.394 5.394 5.398 5.419
    yv       (points) float64 52.24 52.24 52.24 52.24 ... 52.77 52.77 52.77
    gewas    (points) int64 -9223372036854775808 ... -9223372036854775808
Attributes:
    name:     synthetic_test_stm_assendelft

In [34]:
data2 = data2.chunk(chunks={'points':1000,'time':1})

In [35]:
field = xr.DataArray(coords=data2.xv.coords,name='soilcode').astype(str).chunk({'points':1000})
#field = xr.DataArray(coords=data_xr.xv.coords,name='soilcode').astype(str)

In [36]:
field

<xarray.DataArray 'soilcode' (points: 10000)>
dask.array<xarray-<this-array>, shape=(10000,), dtype=<U32, chunksize=(1000,), chunktype=numpy.ndarray>
Coordinates:
  * points   (points) int64 0 1 2 3 4 5 6 ... 9993 9994 9995 9996 9997 9998 9999

In [37]:
data2['soilcode']=field

In [38]:
data2

<xarray.Dataset>
Dimensions:   (points: 10000, time: 3)
Coordinates:
  * points    (points) int64 0 1 2 3 4 5 6 ... 9994 9995 9996 9997 9998 9999
  * time      (time) int64 0 1 2
Data variables:
    h2ph      (points, time) float64 dask.array<chunksize=(1000, 1), meta=np.ndarray>
    xv        (points) float64 dask.array<chunksize=(1000,), meta=np.ndarray>
    yv        (points) float64 dask.array<chunksize=(1000,), meta=np.ndarray>
    gewas     (points) int64 dask.array<chunksize=(1000,), meta=np.ndarray>
    soilcode  (points) <U32 dask.array<chunksize=(1000,), meta=np.ndarray>
Attributes:
    name:     synthetic_test_stm_assendelft

create RTree of geometries to expedite lookup.

NOTE: This currently uses pygeos, however, this functionality will be included in STRtree wwhich will change in shapely 2.0 and greater, to a more powerful imolementation. However the change is breaking 

In [39]:
geom_tree = pygeos.STRtree(as_poly_geoms)

To allow for out of core computation in the enrichment/filtering of the STM we must ensure that the entire dask array is not realized during the execution. To this end it makes sense to employ the dask 'map_blocks' method which would allow the enrichment filtering to be performed chunckwise, thus reducing the memory footprint. We must therefore write a 'numpy-backed' function that can then be applied to each block separately with the aggregated result constituting the full operation. 

At this point it also makes swnse to consider the filtering operation. All points in eaach. block will be comparedd with all the polygons in the list to be considered. If the latter is a large list, and the blocks of each point only cover a reasonably contiguous spatial region, this is unneccessarily wasteful. Instead, it may be beneficial to subselect the polygons to the region covered by the points. of the block

We thus first define the enrichment function. `xrds` represents the xarray `chunk`, `geomTree` the STRtree of all geometries/polygons to be queried, `pygeosgeoms` is the array if geometries in pygeos format, `geoms` is the original geometries object with the relevant vaariables/attributes associated with each geometry, and `geomValFieldName` is the string identifier for the variable/attribute to be enriched. The method assumes the xarray object has been expanded to accommodate the new/enriched field prior to its execution 

In [40]:
def geom_enrich(xrds,geomTree,pygeosgeoms,geoms,geomValFieldName):
              
    xvmax = xrds['xv'].data.max()
    xvmin = xrds['xv'].data.min()
    yvmax = xrds['yv'].data.max()
    yvmin = xrds['yv'].data.min()
    xrdsbbox = pygeos.box(xvmin,yvmin,xvmax,yvmax)
    
    intgeomids = geomTree.query(xrdsbbox,predicate='intersects')
    intpygeosgeoms = pygeosgeoms[intgeomids]
    intgeomFvals = geoms[geomValFieldName][intgeomids]
    
    
    pointTree = pygeos.STRtree(pygeos.points(xrds['xv'].data, xrds['yv'].data))
    intml = np.array(pointTree.query_bulk(intpygeosgeoms, predicate='contains').T.tolist())
    if intml.ndim == 2:
        intuids = np.unique(intml[:,0])
        for i, intuid in enumerate(intuids):
            #print(i, intuid)
            intm = np.where(intml[:,0]==intuid)[0]
            intmid = intml[intm,1]
            xrds[geomValFieldName].data[intmid] = intgeomFvals.iloc[intuid]
            #xrds[geomValFieldName].data[intmid] = intgeomFvals[intgeomids[intuid]]
    else:
        pass
        
    return xrds
    

With the methood definded the lazy mapping can be performed as, where the arguments supplied are specific for this example

In [41]:
data2 = xr.map_blocks(geom_enrich, data2, args=(geom_tree,as_poly_geoms,as_poly,'soilcode'), template=data2)   

Realization can then be triggered as

In [42]:
data2.compute()

<xarray.Dataset>
Dimensions:   (points: 10000, time: 3)
Coordinates:
  * points    (points) int64 0 1 2 3 4 5 6 ... 9994 9995 9996 9997 9998 9999
  * time      (time) int64 0 1 2
Data variables:
    h2ph      (points, time) float64 0.9008 0.1539 0.5195 ... 0.7984 0.1721
    xv        (points) float64 4.313 4.33 4.331 4.369 ... 5.394 5.398 5.419
    yv        (points) float64 52.24 52.24 52.24 52.24 ... 52.77 52.77 52.77
    gewas     (points) int64 -9223372036854775808 ... -9223372036854775808
    soilcode  (points) <U32 'nan' 'nan' 'nan' 'nan' ... 'nan' 'nan' 'nan' 'nan'
Attributes:
    name:     synthetic_test_stm_assendelft